# Tabular Q-learning for the toy problem taxi #

## Taxi ##

There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

Rendering:

* blue: passenger

* magenta: destination

* yellow: empty taxi

* green: full taxi

* other letters: locations

https://gym.openai.com/envs/Taxi-v2/

https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

## Imports ##

In [1]:
import numpy as np
import gym
import random

%matplotlib inline
import matplotlib.pyplot as plt

## Create taxi environment ##

In [2]:
env = gym.make("Taxi-v2")

## Show an initial state ##

In [3]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Create Q table and initialize it ##

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [5]:
print("Number of states:", state_size)
print("Number of actions:", action_size)

Number of states: 500
Number of actions: 6


In [6]:
q_table = np.zeros((state_size, action_size))

## Hyperparameters ##

In [7]:
total_episodes = 100 * 1000   # total episodes
total_test_episodes = 1000    # total test episodes
max_steps = 99                # max steps per episode

learning_rate = 0.1           # learning rate
gamma = 0.95                  # discounting factor

# Parameters for epsilon greedy strategy
epsilon = 1.0                 # Exploration probability
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.0             # Minimum exploration probability 
decay_rate = 0.0001           # Exponential decay rate for exploration prob

## Q-learning algorithm ##

In [ ]:
episode_reward_list = []

for episode in range(total_episodes):
    # reset the environment
    state = env.reset()
    total_reward = 0
    
    for step in range(max_steps):
        # epsilon greedy strategy 
        if random.uniform(0.0, 1.0) <= epsilon:
            # random choice
            action = env.action_space.sample()
        else:
            # select best action in state s
            # a = argmax_a' q(s,a')
            action = np.argmax(q_table[state,:])

        # take action a, get reward r, and observe next_state s'
        new_state, reward, done, _ = env.step(action)
        
        # update q table according to transition (s, a, r, s')
        # q(s,a) := q(s,a) + alpha [r + gamma * max q(s',a') - q(s,a)]
        q_table[state, action] += learning_rate * (reward + gamma * np.max(q_table[new_state, :]) - q_table[state, action])
        
        total_reward += reward
                
        # If done, then finish episode, else update state
        if done: 
            break
        else:
            state = new_state

    if episode % (10 * 1000) == 0: 
        print("Episode {0:6}, total reward {1:6.1f}, epsilon {2:.6f}".format(episode, total_reward, epsilon))
            
    # decrease epsilon 
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode) 
    episode_reward_list.append((episode, total_reward))

Episode      0, total reward -378.0, epsilon 1.000000
Episode  10000, total reward  -33.0, epsilon 0.367916
Episode  20000, total reward  -25.0, epsilon 0.135349
Episode  30000, total reward    3.0, epsilon 0.049792
Episode  40000, total reward    9.0, epsilon 0.018317
Episode  50000, total reward   15.0, epsilon 0.006739
Episode  60000, total reward    8.0, epsilon 0.002479
Episode  70000, total reward    9.0, epsilon 0.000912
Episode  80000, total reward   10.0, epsilon 0.000335
Episode  90000, total reward   12.0, epsilon 0.000123


## Visualize learning progress ##

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N

In [ ]:
eps, rews = np.array(episode_reward_list).T
smoothed_rews = running_mean(rews, 5)
plt.plot(eps[-len(smoothed_rews):], smoothed_rews)
plt.xlabel('Episode')
plt.ylabel('Total Reward')

## Observe trained agent ##

In [ ]:
reward_list = []
for episode in range(total_test_episodes):
    state = env.reset()
    total_reward = 0
    for step in range(max_steps):
        
        # take the best action
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, _ = env.step(action)
        total_reward += reward
        #env.render()
        
        if done:
            reward_list.append(total_reward)
            break
            
        state = new_state        
env.close()

In [ ]:
running_averages = np.convolve(reward_list, np.ones(100,) / 100, mode='valid')

In [ ]:
np.max(running_averages)